Amalies attempt at helping Kverhaug with questions 8, 9 and 10:

Loop for hyperparameter testing and output print statements (made to match ``gradient_descent_output.txt``):

In [ ]:
def calculate_accuracy(y_pred, y_true):
    """ calculates fraction of predictions that are correct """
    good_predictions = (y_pred == y_true)
    acc = np.sum(good_predictions) / len(y_true)
    
    return acc

# for storing which MLP does the best
current_acc = 0
val_accs = []

# parameters to test (same as from output text)
lr = 0.01 # don't tune learning rate
moms = [0, 0, 0.9, 0.9, 0.8]
decays = [0, 0.01, 0, 0.01, 0.001, 0.01]


# use list of models to avoid having to train them so much? 
# not sure how this will work with all the pytorch subclass stuff

for i in range(len(moms)):
    mom = moms[i]
    decay = decays[i]
    
    print(' =========================================================\n  Current parameters:')
    print('lr: %.2f'%lr)
    print('mom: %.2f'%mom)
    print('decay: %.2f'%decays)
    
    # Train your Pytorch's SGD MLP (model1) here and return y_pred_train from last loop of training
    
    
    # calculating accuracies from training
    acc_t = calculate_accuracy(y_pred_train, y_train)
    
    # calculate accuracies from validation set
    y_pred_val = model1(x_validation)
    acc_v = calc_acc(y_pred_val, y_validation)
    if acc_v > current_acc:
        best_mom = mom
        best_decay = decay
        best_SDG = 'PyTorch'
        current_acc = acc_v
    
    print('\n --- Accuracies ---')
    print('\nTraining\nAccuracy: %.2f'%acc_t)
    print('\nValidation\nAccuracy: %.2f\n'%acc_v)
    
    # Train your manual update MLP (model2) here and return y_pred_train from last loop of training
    
    # calculating accuracies from training
    acc_t = calculate_accuracy(y_pred_train, y_train)
    
    # calculate accuracies from validation set
    y_pred_val = model2(x_validation)
    acc_v = calc_acc(y_pred_val, y_validation)
    val_accs.append(acc_v)
    if acc_v > current_acc:
        best_mom = mom
        best_decay = decay
        best_SDG = 'manual update'
        current_acc = acc_v
    
    print('\n --- Accuracies ---')
    print('Training\nAccuracy: %.2f'%acc_t)
    print('Validation\nAccuracy: %.2f\n'%acc_v)

# if we make function this could be used?    
# best_hyperparams = {'momentum' : best_mom, 'decay' : best_decay, 'SGD version' : best_SDG}

Questions 9 and 10:
Select the best model among those trained in the previous question based on their accuracy. Evaluate the best model.

In [ ]:
print('\n --- Best model ---')
print('Learning rate: %.2f'%lr)
print('Momentum: %.2f'%best_mom)
print('Decay: %.2f'%best_decay)
print('Validation accuracy: %.2f'%current_acc)

# train the best model? want no output here
y_pred_eval = model(x_eval)
acc_eval = calculate_accuracy(y_pred_eval, y_eval)

print('Test accuracy: %.2f'%acc_eval)